In [1]:
!pip install transformers

You should consider upgrading via the '/Users/admin/.pyenv/versions/3.7.7/envs/finbert/bin/python -m pip install --upgrade pip' command.


In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [50]:
path = '/Users/admin/code/Michiel-DK/finbert/notebooks/pretrained.pth'

In [54]:
torch.save(model.state_dict(), path)


In [55]:
model.load_state_dict(torch.load(path, map_location=torch.device('cpu')), strict=False)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [51]:
# original saved file with DataParallel
checkpoint = torch.load(path, map_location=torch.device('cpu'))

# create new OrderedDict that does not contain `module.`
from collections import OrderedDict

new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    name = k.replace("module.", "") # remove `module.`
    new_state_dict[name] = v

# load params
model.load_state_dict(new_state_dict, strict=False)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [49]:
model.load_state_dict(torch.load('/Users/admin/code/Michiel-DK/finbert/notebooks/pretrained.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [45]:
!pwd

/Users/admin/code/Michiel-DK/finbert/notebooks


In [44]:
type(tokenizer)

transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

In [2]:
import pandas as pd

#headlines_df = pd.read_csv('../data/analyst_ratings_processed.csv')
#headlines_df = headlines_df.sample(300, random_state=42)

In [14]:
import numpy as np

headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,1])

print(headlines_list[:5])

NameError: name 'headlines_df' is not defined

In [ ]:
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')

In [ ]:
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([300, 3])


In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.9402, 0.0254, 0.0344],
        [0.2727, 0.6773, 0.0500],
        [0.0830, 0.7072, 0.2098],
        [0.9374, 0.0175, 0.0451],
        [0.4195, 0.0178, 0.5627],
        [0.8486, 0.0541, 0.0973],
        [0.0215, 0.2806, 0.6979],
        [0.1797, 0.0092, 0.8111],
        [0.8760, 0.0111, 0.1129],
        [0.7264, 0.0188, 0.2548],
        [0.0858, 0.0994, 0.8148],
        [0.0789, 0.0100, 0.9112],
        [0.7675, 0.0193, 0.2131],
        [0.0080, 0.9702, 0.0218],
        [0.9395, 0.0110, 0.0495],
        [0.8589, 0.1016, 0.0395],
        [0.7818, 0.1004, 0.1177],
        [0.0080, 0.9754, 0.0166],
        [0.0580, 0.2505, 0.6915],
        [0.2982, 0.4383, 0.2635],
        [0.0094, 0.9708, 0.0198],
        [0.9154, 0.0385, 0.0461],
        [0.0365, 0.0528, 0.9107],
        [0.0282, 0.6327, 0.3391],
        [0.0243, 0.1238, 0.8520],
        [0.0350, 0.0672, 0.8978],
        [0.0246, 0.6180, 0.3574],
        [0.0368, 0.1453, 0.8179],
        [0.0238, 0.1664, 0.8099],
        [0.009

In [ ]:
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

In [ ]:
df.sort_values(by='Neutral', ascending=False)

,Headline,Positive,Negative,Neutral
271,Annual Shareholder Meetings Being Held Today By: Fiserv and MGP Ingredients,0.023845,0.020296,0.955859
43,Sequential Brands Reports Launch of Office by Martha Stewart Exclusively at Staples,0.028870,0.019753,0.951377
250,"UBS Research Summary (FLEX, ARW, BHI, BKI, CTXS, DPS, EGN, ESRX, ISIL, LIS, PH, TMO, UHS, WLP, XLNX, NSC)",0.023946,0.026518,0.949535
77,"Amazon Is Taking Over The Market, One Sector At A Time",0.041424,0.012997,0.945579
109,"Benzinga’s Volume Movers (BARE, MAPP, HWCC, LEAP, IART)",0.039839,0.014831,0.945331
45,"Zacks Industry Outlook Highlights: UnitedHealth Group, Humana, HealthSpring and Universal American - Press Releases",0.029708,0.025361,0.944931
139,Shares of Broadsoft to Resume Trade at 4:35PM EST,0.040232,0.016843,0.942925
182,This ETF Is No Turkey,0.027637,0.030163,0.942199
58,Hearing Prescience Point Has Issued Short Idea On Kellogg; BZ NOTE: Firm Has Not Yet Tweeted On Stock,0.021328,0.036705,0.941967
161,"Benzinga's M&A Chatter for Friday September 28, 2012",0.021499,0.036781,0.941720


In [16]:
# Read JSON
import json
from datetime import datetime
#json_file_path = f"../data/{datetime.today().strftime('%Y-%m-%d')}.json"
json_file_path = "../data/2021-11-21.json"


with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

In [17]:
contents['HALO']

{'news': [['Nov-18-21',
   'Halozyme to Present at Upcoming Investor Conferences'],
  ['Nov-05-21',
   "Here's Why I Think Halozyme Therapeutics (NASDAQ:HALO) Is An Interesting Stock"],
  ['Nov-03-21',
   "Halozyme's Q3 Sales Boosted By Darzalex; Lifts FY21 Earnings Outlook"],
  ['Nov-03-21', 'Halozyme (HALO) Beats on Q3 Earnings, Tweaks 2021 Guidance'],
  ['Nov-02-21',
   'Halozyme Therapeutics (HALO) Q3 Earnings and Revenues Top Estimates'],
  ['Nov-02-21', 'Halozyme Reports Third Quarter 2021 Results'],
  ['Nov-01-21', 'Should You Buy Halozyme (HALO) Ahead of Earnings?'],
  ['Oct-28-21',
   'Why the Earnings Surprise Streak Could Continue for Halozyme Therapeutics (HALO)'],
  ['Oct-26-21',
   'Halozyme Therapeutics, Inc. (HALO) Fell Out Of Favor With Hedge Funds'],
  ['Oct-26-21',
   'Halozyme Therapeutics (HALO) Earnings Expected to Grow: Should You Buy?'],
  ['Oct-19-21',
   'Halozyme To Host Third Quarter 2021 Financial Results Webcast and Conference Call'],
  ['Oct-05-21',
   'N

In [56]:
new_ls = []
#for ticker in contents.keys():
for ticker in ['HALO', 'OLED']:
    for item in contents[ticker]['news']:
        item.append(ticker)
        new_ls.append(item)
        

In [57]:
round(len(new_ls)/300)

1

In [20]:
import numpy as np
headlines_array = np.array(new_ls)
np.random.shuffle(headlines_array)


In [21]:
headlines_array[0]

array(['Jul-19-21',
       'Hedge Funds Are Cashing Out Of Halozyme Therapeutics, Inc. (HALO)',
       'HALO'], dtype='<U257')

In [58]:
from tqdm import tqdm
import torch
import pandas as pd

df = pd.DataFrame(columns = ["date", "ticker","headline", "positive", "negative", "neutral"])

for group in tqdm(np.array_split(headlines_array, round(len(headlines_array)/300))):
    headlines_list = list(group[:,1])
    ticker_list = list(group[:,-1])
    date_list = list(group[:,0])

    inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()

    table = {'date':date_list,
        'ticker':ticker_list,
        'headline':headlines_list,
            "positive":positive,
            "negative":negative, 
            "neutral":neutral}
    
    df = df.append(pd.DataFrame.from_dict(table))
df = df.reset_index(drop=True)

100%|██████████| 1/1 [02:42<00:00, 162.89s/it]


In [53]:
df

,date,ticker,headline,positive,negative,neutral
0,Jul-19-21,HALO,Hedge Funds Are Cashing Out Of Halozyme Therap...,0.040041,0.291833,0.668126
1,Nov-14-21,OLED,Thinking About Shiba Inu? Buy These 3 Stocks I...,0.046935,0.022655,0.930411
2,Aug-26-21,HALO,Halozyme to Present at Wells Fargo 2021 Virtua...,0.048928,0.014682,0.936390
3,Jul-30-21,HALO,Halozyme to Participate in Canaccord Genuity 4...,0.188573,0.007553,0.803874
4,Feb-18-21,OLED,Universal Display Corporation Increases Quarte...,0.762251,0.117014,0.120735
...,...,...,...,...,...,...
195,Oct-05-21,HALO,Notable Halozyme Therapeutics Insider Trades $...,0.030459,0.021451,0.948091
196,Oct-06-21,OLED,Estimating The Fair Value Of Universal Display...,0.062149,0.015094,0.922757
197,Nov-05-21,OLED,Universal Display stock after earnings hint at...,0.068910,0.911137,0.019954
198,Apr-20-21,HALO,Halozyme To Host First Quarter 2021 Financial ...,0.041097,0.014765,0.944138


In [21]:
df.groupby('ticker').mean().sort_values(by='positive')

,positive,negative,neutral
ticker,,,
OLED,0.236796,0.128001,0.635203
HALO,0.352239,0.108665,0.539095


In [5]:
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
outputs = model(**inputs)


In [ ]:
print(outputs.logits.shape)

In [ ]:
import torch
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [ ]:
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Ticker':ticker_list,
    'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Ticker","Headline", "Positive", "Negative", "Neutral"])

In [ ]:
df